# Question 9


Build a Python Cashflow Model that allows the user to forecasts expected cashflows on a loan-by-loan basis using input CPR / CDR and Recovery Curves. A simple excel example has been given on the "Simple Mortgage Model" tab

In [56]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from portfolio_model import Loans_Portfolio

In [56]:
mystyle = '../style/pastel-light.mplstyle'
plt.style.use(mystyle)
%config InlineBackend.figure_format='retina'

pd.options.mode.copy_on_write = True 
pd.options.display.float_format = "{:,.2f}".format

In [57]:
cob = datetime.date(2022, 12, 31)  # 31/12/2022
ss_path = '../data/2024_Strat_Casestudy.xlsx'

portfolio = Loans_Portfolio(cob_date=cob, data_path=ss_path)

In [59]:
CPR = portfolio.construct_portfolio_cpr(index='time_to_reversion')
CDR = portfolio.construct_portfolio_cdr(index='time_to_reversion')
inputs_for_model = CPR.merge(CDR, left_index=True, right_index=True)
inputs_for_model.head()

,smm,cpr,mdr,cdr
-24,0.00,0.02,0.00,0.00
-23,0.00,0.01,0.00,0.00
-22,0.00,0.00,0.00,0.02
-21,0.00,0.01,0.00,0.01
-20,0.00,0.03,0.00,0.02


In [61]:
inputs_for_model.index

Index([-24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11,
       -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,
         4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
      dtype='int64')

In [62]:
loan_data = {
    'Months_Post_Reversion' : -22,
    'Seasoning' : 2,
    'current_balance' : 100000,
    'Fixed Pre-Reversion Rate':	0.0394,
    'Post Reversion Margin': 	0.0494,
    'Months to Maturity':	178,
    'repayment_method':	'Interest Only'
}

model_inputs = { 'loan_data' : loan_data,
                 'CDR' : inputs_for_model.cdr.reset_index(drop=True),
                 'CPR' : inputs_for_model.cpr.reset_index(drop=True),
                 'months_for_reversion' : inputs_for_model.index,
                }

In [67]:
n_forecast = 200 
forecast_boe_base_rate = np.full(n_forecast, 0.02)  # Fixed for now

In [72]:
opening_balance = np.zeros(n_forecast)
interest_rate = np.zeros(n_forecast)
remaining_term = np.zeros(n_forecast)
time_past_reversion = np.zeros(n_forecast)
scheduled_interest = np.zeros(n_forecast)
scheduled_payment = np.zeros(n_forecast)
scheduled_principal = np.zeros(n_forecast)
closing_balance = np.zeros(n_forecast)

for i in range(n_forecast):

    if i == 0:
        opening_balance[i] == model_inputs['loan_data'].get('current_balance')
    else:
        opening_balance[i] = closing_balance[i-1]


    remaining_term[i] = np.max(model_inputs['loan_data'].get('Months to Maturity') - 1, 0)
    time_past_reversion[i] = model_inputs['loan_data'].get(
        'Months_Post_Reversion')
    
    if time_past_reversion[i] > 0 :
        interest_rate[i] = forecast_boe_base_rate[i] + model_inputs['loan_data'].get(
            'Post Reversion Margin')
    else:
        interest_rate[i] = model_inputs['loan_data'].get(
            'Fixed Pre-Reversion Rate')

    scheduled_interest[i] = opening_balance[i]*(interest_rate[i]/12)
    scheduled_payment[i] = scheduled_interest[i]
    # scheduled_principal[i] = 0 for now only interests
    closing_balance[i] = opening_balance[i] - scheduled_principal[i]
    

In [53]:
EOP = np.zeros(N)
Defaults = np.zeros(N)
ExpBPPD = np.zeros(N)
ExpBPPP = np.zeros(N)
EP = np.zeros(N)
ECPB = np.zeros(N)


In [54]:
for i in range(N):
    if i == 0:
        EOP[i] = balance0
    else:
        EOP[i] = ECPB[i-1]

    Defaults[i] = (1. - (1. - CDR[i])**(1/12))*EOP[i]
    ExpBPPD[i] = EOP[i] - Defaults[i]
    ExpBPPP[i] = ExpBPPD[i] - 0. # Only interests for now
    EP[i] = (1. - (1. - CPR[i])**(1/12))*ExpBPPP[i]
    ECPB[i] = ExpBPPP[i] - EP[i]

In [55]:
pd.DataFrame({'cdr': CDR, 'cpr': CPR, 'Expected_Opening_Perfornace': EOP,
                 'Expected Closing Default Balance': ECPB,
                 'Defaults': Defaults,
                 'Expected Balance Post Period Defaults': ExpBPPP,
                 'Expected Balance Pre Period Prepays': ExpBPPP,
                 'Expected Prepayments': EP
                 })

,cdr,cpr,Expected_Opening_Perfornace,Expected Closing Default Balance,Defaults,Expected Balance Post Period Defaults,Expected Balance Pre Period Prepays,Expected Prepayments
0,0.00,0.02,"100,000.00","99,817.90",0.00,"100,000.00","100,000.00",182.10
1,0.00,0.01,"99,817.90","99,765.36",0.00,"99,817.90","99,817.90",52.53
2,0.02,0.00,"99,765.36","99,526.82",202.67,"99,562.70","99,562.70",35.87
3,0.01,0.01,"99,526.82","99,370.32",55.03,"99,471.79","99,471.79",101.48
4,0.02,0.03,"99,370.32","98,922.03",164.31,"99,206.01","99,206.01",283.98
...,...,...,...,...,...,...,...,...
79,0.00,0.42,"35,741.31","34,137.41",0.00,"35,741.31","35,741.31","1,603.91"
80,0.00,0.00,"34,137.41","34,137.41",0.00,"34,137.41","34,137.41",0.00
81,0.00,0.00,"34,137.41","34,137.41",0.00,"34,137.41","34,137.41",0.00
82,0.00,0.00,"34,137.41","34,137.41",0.00,"34,137.41","34,137.41",0.00


In [ ]:
CPR = 0.02
CDR = 0.02

balance0 = 100000

eop = balance0
defaults = (1. - (1. - CDR)**(1/12))*eop
ebppd =  eop - defaults
ebppp = ebppd - 0.


ep = (1. - (1. - CPR)**(1/12))*ebppp

ecpb = ebppd - ep 
ecpb 